In [1]:
import os, itertools
import cupy as cp
import numpy as np
from scipy.stats import norm
from numba import jit, njit, prange
from tqdm import trange, tqdm

## Routines

#### Learning rules

In [2]:
class BilinearPlasticityRule:
    def __init__(self):
        self.f = lambda x:x
        self.g = lambda x:x
        
class ThresholdPlasticityRule:
    def __init__(self,
            x_f,
            q_f,
            x_g=None,
            q_g=None,
            rv=norm):
        if not x_g: x_g = x_f
        if not q_g: q_g = rv.cdf(x_g)
        self.x_f, self.q_f = x_f, q_f
        self.x_g, self.q_g = x_g, q_g
        def f(x):
            return np.where(x > x_f, q_f, -(1-q_f))
        def g(x):
            return np.where(x > x_g, q_g, -(1-q_g))
        self.f = f
        self.g = g

#### Structural connectivity

In [3]:
def build_ji(k, N):
    ji = []
    arr = np.arange(N)
    for i in trange(N):
        tmp = np.concatenate((arr[:i], arr[i+1:]))
        j_subset = np.random.choice(N, size=k[i], replace=False)
        ji.append(np.asarray(sorted(j_subset)))
    return ji

#### Storing associations

In [5]:
def weight(seq, p, f, g, i, j):
    "$f(xi_i^{\mu+p}) * g(xi_j^{\mu})$"
    if p == 0:
        return np.sum(f(seq[:,i][:,np.newaxis]) * g(seq[:,j]), axis=0)
    else:
        return np.sum(f(seq[p:,i][:,np.newaxis]) * g(seq[:-p,j]), axis=0)

def store_associations(seq, f, g, ji, K, p=1):
    "outputs in csr representation"
    N = len(ji)
    indptr = np.asarray([0]+[len(ji[i]) for i in range(N)]).cumsum()
    indices = np.concatenate(ji)
    data = []
    for i in trange(N):
        j = ji[i]
        w = np.asarray(weight(seq, p, f, g, i, j) / K, np.float32)
        data.extend(w)
    return indptr, indices, data

#### Partition into subpopulations

In [6]:
def reweight(data, A, N, w_11, w_12, w_21, w_22):
    N1 = int(N/2)
    for n, (idx_0, idx_1) in tqdm(enumerate(zip(indptr[:N], indptr[1:N+1]))):
        idxs = indices[idx_0:idx_1]
        idx_m = idx_0 + idxs[idxs < N1].size
        if n < N1:
            data[idx_0:idx_m] *= A*w_11
            data[idx_m:idx_1] *= A*w_12
        else:
            data[idx_0:idx_m] *= A*w_21
            data[idx_m:idx_1] *= A*w_22

#### Simulate

In [7]:
def erf(x):
    z = abs(x)
    t = 1.0/(1.0+0.5*z)
    ans = t*cp.exp(-z*z-1.26551223+t*(1.00002368+t*(0.37409196+t*(0.09678418+ \
        t*(-0.18628806+t*(0.27886807+t*(-1.13520398+t*(1.48851587+ \
        t*(-0.82215223+t*0.17087277)))))))))
    return 1 - cp.where(x > 0, ans, 2.0-ans)

def phi(x, theta, sigma):
    return 0.5 * (1 + erf((x - theta) / (cp.sqrt(2) * sigma)))

def simulate_euler(t0, T, dt, tau, r0, W, theta, sigma, I_ext=lambda x: 0, disable_pbar=False):
    "$\frac{dx}{dt} = -x + \phi( \sum_{j} J_{ij} x_j + I_0 )$"
    state = cp.zeros((N, int((T-t0)/dt)))
    state[:,0] = r0
    for i, t in enumerate(tqdm(np.arange(t0,T-dt,dt), disable=disable_pbar)):
        r = state[:,i]
        r_sum = W.dot(r) + I_ext(t)
        dr = (-r + phi(r_sum, theta, sigma)) / tau
        state[:,i+1] = r + dt * dr
    return state

In [8]:
def simulate(T, dt, tau, I_ext_1, I_ext_2, theta, sigma, patterns, W, disable_pbar=False):
    def I_ext(t):
        return cp.concatenate((
            cp.full(shape=(int(N/2)), fill_value=I_ext_1, dtype=cp.float32), # -0.8
            cp.full(shape=(int(N/2)), fill_value=I_ext_2, dtype=cp.float32))) # 0
        
    r0=phi(cp.asarray(patterns[0,:]), theta, sigma)
        
    r = simulate_euler(
        0, T, dt,
        tau,
        r0,
        W,
        theta, sigma,
        I_ext,
        disable_pbar)
    return r.get()

#### Correlations

In [9]:
xp = cp
def correlations(r, patterns, individual=False):
    "Assumes populations of equal size"
    P, N, T = patterns.shape[0], r.shape[0], r.shape[1]
    n1 = int(N/2)
    n2 = int(N/2)
    r = xp.asarray(r)
    q = xp.zeros(shape=(P,T)) 
    q1 = xp.zeros(shape=(P,T))
    q2 = xp.zeros(shape=(P,T))
    for u, pattern in enumerate(xp.asarray(patterns)):
        # q: Correlation of whole population
        pattern_mean = pattern.mean()
        pattern_std = xp.sqrt(xp.sum((pattern-pattern_mean)**2))
        for t in range(T):
            q[u,t] = xp.sum((pattern - pattern_mean) * (r[:,t] - r[:,t].mean())) / \
                xp.sqrt(xp.sum((r[:,t]-r[:,t].mean())**2)) / \
                pattern_std
        if individual:
            # q_1: Population 1 correlations
            pattern_mean = pattern[:n1].mean()
            pattern_std = xp.sqrt(xp.sum((pattern[:n1]-pattern_mean)**2))
            for t in range(T):
                q1[u,t] = xp.sum((pattern[:n1] - pattern_mean) * (r[:n1,t] - r[:n2,t].mean())) / \
                    xp.sqrt(xp.sum((r[:n1,t]-r[:n1,t].mean())**2)) / \
                    pattern_std
            # q_2: Population 2 correlations
            pattern_mean = pattern[n1:].mean()
            pattern_std = xp.sqrt(xp.sum((pattern[n1:]-pattern_mean)**2))
            for t in range(T):
                q2[u,t] = xp.sum((pattern[n1:] - pattern_mean) * (r[n1:,t] - r[n1:,t].mean())) / \
                    xp.sqrt(xp.sum((r[n1:,t]-r[n1:,t].mean())**2)) / \
                    pattern_std
    return q.get(), q1.get(), q2.get()

## Figure data generation

In [10]:
N = 80000
p = 0.005
K = N*p
P = 16
tau = 0.01
dt = 1e-3

A = 20
theta = 0
sigma = 0.05
x_f = 1.5
q_f = 0.8

In [11]:
patterns = np.random.RandomState(seed=1).normal(0,1,size=(P,N))

In [12]:
# Structural connectivity
k = np.random.RandomState(seed=2).binomial(N, p, size=N)
k[:] = 400 # fixed degree
ji = build_ji(k, N)

100%|██████████| 80000/80000 [01:47<00:00, 746.47it/s]


In [13]:
# Store pattern associations using the threshold plasticity rule
plasticity = ThresholdPlasticityRule(x_f, q_f)
indptr, indices, data_p0 = store_associations(
    patterns, plasticity.f, plasticity.g, ji, K, p=0)
_, _, data_p1 = store_associations(
    patterns, plasticity.f, plasticity.g, ji, K, p=1)

100%|██████████| 80000/80000 [00:07<00:00, 10578.76it/s]


In [14]:
# Adjust subpopulation weights
data_p1_copy = np.asarray(data_p1).copy()
data_p0_copy = np.asarray(data_p0).copy()
reweight(data_p1_copy, 
         A, N, w_11=1, w_12=1, w_21=0, w_22=0)
reweight(data_p0_copy,
         A, N, w_11=0, w_12=0, w_21=1, w_22=1)
data = data_p1_copy + data_p0_copy
W = cp.sparse.csr_matrix(
    (cp.asarray(data), cp.asarray(indices), cp.asarray(indptr)),
    shape=(N,N),
    dtype=cp.float32)

80000it [00:00, 147606.66it/s]
80000it [00:00, 148345.83it/s]


### Simulation runner

In [ ]:
I_ext_1=np.arange(-0.6,-0.8,-0.6/39)[::-1]
I_ext_2=np.linspace(-0.4,0.,40)

params = {
    "N": N,
    "p": p,
    "K": K,
    "P": P,
    "A": A,
    "tau": tau,
    "theta": theta,
    "sigma": sigma,
    "x_f": plasticity.x_f,
    "q_f": plasticity.q_f,
    "x_g": plasticity.x_g,
    "q_g": plasticity.q_g,
}

combinations = list(itertools.product(
    np.atleast_1d(I_ext_1),
    np.atleast_1d(I_ext_2)))

directory = "3-nonlinear-phase-diagram/data/"

try:
    os.makedirs(directory)
except FileExistsError:
    pass

for args in tqdm(combinations[:]):
    params["I_ext_1"] = I_ext_1 = args[0]
    params["I_ext_2"] = I_ext_2 = args[1]
    
    r = simulate(
        2.0,  # T
        1e-3, # dt
        0.01, # tau
        I_ext_1,
        I_ext_2,
        theta,
        sigma,
        patterns,
        W,
        disable_pbar=True)
    q, q1, q2 = correlations(r, patterns, individual=True)
    filename = "Iext1%.6f_Iext2%.6f"%(I_ext_1,I_ext_2) + ".npy"
    filepath = directory + filename
    np.save(open(filepath, 'wb'), {
        "q": q,
        "q1": q1,
        "q2": q2,
        "params": params})

 36%|███▌      | 201/560 [1:46:36<3:08:49, 31.56s/it]